### All the required imports are placed here

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Import the CIFAR10 dataset

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, 
                                              shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                            shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Neural Network Implementation

In [3]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        
        self.conv_part = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2)
        )
        
        self.connected_layers = nn.Sequential(
            nn.Linear(6 * 6 * 256, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 10)
        )
        
    
    def forward(self, x):
        x = self.conv_part(x)
        x = x.view(-1, 6 * 6 * 256)
        x = nn.connected_layers(x)
        
        return x

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convolve = nn.Sequential(
            nn.Conv2d(3, 64, 5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(1, 4),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(1, 2)
        )
        
        self.connected = nn.Sequential(
            nn.Linear(5 * 5 * 128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.convolve(x)
        print(x.size)
        x = x.view(-1, 128 * 5 * 5)
        x = self.connected(x)
        
        return x

net = Net()
net = net.cuda()

# Loss Function and Optimizer
- Using the cross entropy loss function
- Using Gradient Descent with Adam

In [34]:
learning_rate = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Training Loop

In [35]:
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        #Forward Prop
        outputs = net(inputs)
        #Compute the loss for current batch
        loss = criterion(outputs, labels)
        #Back Prop
        loss.backward()
        #Gradient Descent
        optimizer.step()
        
        #print statistics
        running_loss += loss.data[0]
        if i % 100 == 99:
            print('[Epoch:{} {}] Loss: {}'.format(
                   epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0

print('Finished Training')

<built-in method size of torch.cuda.FloatTensor object at 0x7f3a53fd8c08>


RuntimeError: invalid argument 2: size '[-1 x 3200]' is invalid for input with 131072 elements at /opt/conda/conda-bld/pytorch_1512387374934/work/torch/lib/TH/THStorage.c:37

In [28]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: {} %%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 54.97 %%
